In [1]:
!wget https://tcd.miovision.com/static/dataset/MIO-TCD-Localization.tar
!tar -xf MIO-TCD-Localization.tar

--2025-11-17 06:40:40--  https://tcd.miovision.com/static/dataset/MIO-TCD-Localization.tar
Resolving tcd.miovision.com (tcd.miovision.com)... 65.9.95.124, 65.9.95.71, 65.9.95.9, ...
Connecting to tcd.miovision.com (tcd.miovision.com)|65.9.95.124|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3739303424 (3.5G) [application/x-tar]
Saving to: ‘MIO-TCD-Localization.tar’

MIO-TCD-Localizatio 100%[===================>]   3.48G   178MB/s    in 20s     

2025-11-17 06:41:01 (175 MB/s) - ‘MIO-TCD-Localization.tar’ saved [3739303424/3739303424]



In [3]:
!tar -xf MIO-TCD-Localization.tar

In [7]:
!ls /workspace/MIO-TCD-Localization/images/train/ | wc -l

ls: cannot access '/workspace/MIO-TCD-Localization/images/train': No such file or directory
0


In [13]:
!pip install scikit-learn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 150.4 MB/s eta 0:00:00 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 308.4/308.4 kB 92.4 MB/s eta 0:00:00


In [3]:
import pandas as pd
import cv2
import os
from pathlib import Path
from tqdm.notebook import tqdm
import sklearn.model_selection
import yaml
# Assumed class list from your 'localization_evaluation.py'
# This mapping is CRITICAL.
CLASS_NAMES = [
    'articulated_truck', 'bicycle', 'bus', 'car', 'motorcycle',
    'motorized_vehicle', 'non-motorized_vehicle', 'pedestrian',
    'pickup_truck', 'single_unit_truck', 'work_van'
]
CLASS_MAP = {name: i for i, name in enumerate(CLASS_NAMES)}

# --- 2. DEFINE YOUR PATHS ---
# Make sure these paths are correct for your runtime
BASE_DIR = Path("/workspace/MIO-TCD-Localization")
CSV_FILE = BASE_DIR / "gt_train.csv"
IMAGE_DIR = BASE_DIR / "train"  # Folder with all the .jpg images

# Define output directories
TRAIN_LABEL_DIR = BASE_DIR / "labels/train"
VAL_LABEL_DIR = BASE_DIR / "labels/val"
YAML_OUTPUT_PATH = BASE_DIR / "mio_tcd.yaml"

# Define the train/validation split ratio
VAL_SIZE = 0.05  # 5% for validation, 95% for training
RANDOM_STATE = 42

# Create the output directories
TRAIN_LABEL_DIR.mkdir(parents=True, exist_ok=True)
VAL_LABEL_DIR.mkdir(parents=True, exist_ok=True)

# Helper function to convert (x1, y1, x2, y2) to YOLO format
def convert_to_yolo(img_w, img_h, bbox):
    """
    Converts (x1, y1, x2, y2) format to (x_center, y_center, width, height)
    normalized from 0 to 1.
    """
    x1, y1, x2, y2 = bbox

    dw = 1.0 / img_w
    dh = 1.0 / img_h

    x_center = ((x1 + x2) / 2.0) * dw
    y_center = ((y1 + y2) / 2.0) * dh
    width = (x2 - x1) * dw
    height = (y2 - y1) * dh

    return [x_center, y_center, width, height]

# --- 3. READ, SPLIT, AND PROCESS THE CSV ---

print(f"Reading {CSV_FILE}...")
df = pd.read_csv(CSV_FILE, header=None, names=["img_name", "class", "x1", "y1", "x2", "y2"])

# Convert img_name to int then to zero-padded string (8 digits)
df["img_name"] = df["img_name"].astype(int).astype(str).str.zfill(8)

all_image_names = df['img_name'].unique()

print(f"Found {len(all_image_names)} unique images.")

# Split the unique image names into training and validation sets
train_names, val_names = sklearn.model_selection.train_test_split(
    all_image_names,
    test_size=VAL_SIZE,
    random_state=RANDOM_STATE
)

print(f"Splitting data: {len(train_names)} train, {len(val_names)} val")

# Create a dictionary for fast lookup
df_lookup = df.set_index("img_name")

# Function to process a list of image names (for either train or val)
def process_images(image_list, output_dir):
    print(f"Processing {len(image_list)} images for {output_dir.name} set...")

    for img_name in tqdm(image_list):
        # Construct paths
        img_path = IMAGE_DIR / f"{img_name}.jpg"
        label_path = output_dir / f"{img_name}.txt"

        # Read image to get its *actual* dimensions
        try:
            image = cv2.imread(str(img_path))
            img_h, img_w, _ = image.shape
        except Exception as e:
            print(f"\n[Warning] Could not read image {img_path}. Skipping. Error: {e}")
            continue

        # Get all rows for this image
        try:
            image_data = df_lookup.loc[[img_name]]
        except KeyError:
            print(f"\n[Warning] No labels found for {img_name} in CSV. Skipping.")
            continue

        yolo_labels = []
        # Handle single-row (Series) vs multi-row (DataFrame)
        if isinstance(image_data, pd.Series):
             image_data = image_data.to_frame().T

        for _, row in image_data.iterrows():
            class_name = row["class"]

            if class_name not in CLASS_MAP:
                print(f"\n[Warning] Unknown class: {class_name}. Skipping.")
                continue
            class_id = CLASS_MAP[class_name]

            bbox = [row["x1"], row["y1"], row["x2"], row["y2"]]
            yolo_bbox = convert_to_yolo(img_w, img_h, bbox)

            yolo_labels.append(f"{class_id} {' '.join(map(str, yolo_bbox))}")

        # Write all labels for this image to its new .txt file
        with open(label_path, 'w') as f:
            f.write("\n".join(yolo_labels))

# Process both sets
process_images(train_names, TRAIN_LABEL_DIR)
process_images(val_names, VAL_LABEL_DIR)

print("\n--- Preprocessing Finished ---")

# --- 4. GENERATE THE YAML FILE ---

print(f"Generating {YAML_OUTPUT_PATH}...")

# We need to find the relative paths for the YAML file
# The YAML file is at BASE_DIR. The images are in IMAGE_DIR.
# We need to find the path from BASE_DIR to IMAGE_DIR.
# Since IMAGE_DIR is BASE_DIR / "train", the relative path is just "train"
# But we must create relative paths for train/val *images*
# The preprocessing script assumes all images are in ONE folder: IMAGE_DIR.
# YOLOv8 is smart enough to find them.

# We must adjust the YAML to point to the correct *image* folders.
# The MIO-TCD dataset has all images in ONE `train` folder.
# We must create new `images/train` and `images/val` folders
# and *move* the images to match the new label split.

print("Organizing image files into train/val folders...")

# Create new image directories
TRAIN_IMG_DIR = BASE_DIR / "images/train"
VAL_IMG_DIR = BASE_DIR / "images/val"
TRAIN_IMG_DIR.mkdir(parents=True, exist_ok=True)
VAL_IMG_DIR.mkdir(parents=True, exist_ok=True)

# Move the image files to match the label split
for img_name in tqdm(train_names, desc="Moving train images"):
    src_path = IMAGE_DIR / f"{img_name}.jpg"
    dst_path = TRAIN_IMG_DIR / f"{img_name}.jpg"
    if src_path.exists():
        src_path.rename(dst_path)

for img_name in tqdm(val_names, desc="Moving val images"):
    src_path = IMAGE_DIR / f"{img_name}.jpg"
    dst_path = VAL_IMG_DIR / f"{img_name}.jpg"
    if src_path.exists():
        src_path.rename(dst_path)

print("Image files moved.")
print(f"Generating new {YAML_OUTPUT_PATH}...")


Reading /workspace/MIO-TCD-Localization/gt_train.csv...
Found 110000 unique images.
Splitting data: 104500 train, 5500 val
Processing 104500 images for train set...


  0%|          | 0/104500 [00:00<?, ?it/s]

Processing 5500 images for val set...


  0%|          | 0/5500 [00:00<?, ?it/s]


--- Preprocessing Finished ---
Generating /workspace/MIO-TCD-Localization/mio_tcd.yaml...
Organizing image files into train/val folders...


Moving train images:   0%|          | 0/104500 [00:00<?, ?it/s]

Moving val images:   0%|          | 0/5500 [00:00<?, ?it/s]

Image files moved.
Generating new /workspace/MIO-TCD-Localization/mio_tcd.yaml...


In [4]:
CLASS_NAMES = [
    'articulated_truck', 'bicycle', 'bus', 'car', 'motorcycle',
    'motorized_vehicle', 'non-motorized_vehicle', 'pedestrian',
    'pickup_truck', 'single_unit_truck', 'work_van'
]
CLASS_MAP = {name: i for i, name in enumerate(CLASS_NAMES)}

# id -> name for YAML
id2name = {i: name for name, i in CLASS_MAP.items()}

# Define the YAML structure
yaml_data = {
    'path': str(BASE_DIR),
    'train': str(BASE_DIR) + '/images/train',
    'val': str(BASE_DIR) + '/images/val',
    'nc': len(CLASS_NAMES),
    'names': CLASS_NAMES
}

# Write the YAML file
with open(YAML_OUTPUT_PATH, 'w') as f:
    yaml.dump(yaml_data, f, sort_keys=False, default_flow_style=False)

print("\n--- YAML File Generated Successfully ---")
print(f"File created at: {YAML_OUTPUT_PATH}")
print("\nContents:")
print(yaml.dump(yaml_data, sort_keys=False, default_flow_style=False))
print("\nYour dataset is now ready for YOLO training.")


--- YAML File Generated Successfully ---
File created at: /workspace/MIO-TCD-Localization/mio_tcd.yaml

Contents:
path: /workspace/MIO-TCD-Localization
train: /workspace/MIO-TCD-Localization/images/train
val: /workspace/MIO-TCD-Localization/images/val
nc: 11
names:
- articulated_truck
- bicycle
- bus
- car
- motorcycle
- motorized_vehicle
- non-motorized_vehicle
- pedestrian
- pickup_truck
- single_unit_truck
- work_van


Your dataset is now ready for YOLO training.


In [5]:
import os

cores = os.cpu_count()
print(f"Number of CPU cores: {cores}")

Number of CPU cores: 512


In [6]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 37.2 MB/s eta 0:00:00


In [2]:
from ultralytics import YOLO
import pandas as pd
from pathlib import Path
import os

# --- 1. Define Paths ---
DATASET_ROOT = Path("/workspace/MIO-TCD-Localization")
DATASET_YAML = DATASET_ROOT / "mio_tcd.yaml"
DATA_YAML = str(DATASET_YAML)
PROJECT   = "/content/mio_tcd_runs"
IMG_SIZE  = 640
EPOCHS    = 300
WORKERS   = 64

# --- 2. Define Models ---
# We use fresh pretrained models (e.g., yolov12x.pt)
# because this is a new dataset.
models = [
    # ("/content/mio_tcd_runs/yolov8x_MIO/last.pt",  "yolov8x_MIO"),
    # ("/content/mio_tcd_runs/yolo11x_MIO/last.pt", "yolo11x_MIO"),
    ("yolo12x.pt", "yolo12x_MIO"),
]

# --- 3. Define Common Settings ---
common = dict(
    data=DATA_YAML,
    imgsz=IMG_SIZE,
    epochs=EPOCHS,
    batch=-1,        # auto-batch
    device=0,
    optimizer="AdamW",
    cos_lr=True,
    lr0=0.001,
    close_mosaic=10,
    amp=False,
    weight_decay=0.0005,
    patience=15,
    workers=WORKERS,
    cache=True,      # Cache to RAM
    project=PROJECT,
    exist_ok=True,
    # save_period = 10,
    # resume=True,
)

# --- 4. Run Training Loop ---
rows, best_ckpts = [], {}
for w, tag in models:
    print(f"\n=== Training {tag} on MIO-TCD ===")
    try:
        model = YOLO(w)

        # Start new training
        model.train(name=tag, **common)

        # Validate the model
        m = model.val(data=DATA_YAML, imgsz=IMG_SIZE)

        rows.append({"model": tag,
                     "mAP50-95": float(m.box.map),
                     "mAP50": float(m.box.map50),
                     "mAP75": float(m.box.map75)})

        best_ckpts[tag] = str(Path(model.ckpt_path))
        print(f"[OK] {tag} best: {best_ckpts[tag]}")

    except Exception as e:
        print(f"[SKIP] {tag}: {e}")

# --- 5. Save Results ---
df = pd.DataFrame(rows).sort_values("mAP50-95", ascending=False)
cmp_csv = Path(PROJECT) / "model_comparison_MIO.csv"
cmp_csv.parent.mkdir(parents=True, exist_ok=True)
df.to_csv(cmp_csv, index=False)

print("\n--- MIO-TCD Training Complete ---")
print(df)
print(f"Saved comparison CSV to: {cmp_csv}")


=== Training yolo12x_MIO on MIO-TCD ===
New https://pypi.org/project/ultralytics/8.3.229 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.228 🚀 Python-3.12.11 torch-2.9.1+cu128 CUDA:0 (NVIDIA H200 NVL, 143156MiB)
engine/trainer: agnostic_nms=False, amp=False, augment=False, auto_augment=randaugment, batch=-1, bgr=0.0, box=7.5, cache=True, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=True, cutmix=0.0, data=/workspace/MIO-TCD-Localization/mio_tcd.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=300, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.001, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo12x.pt, momentum=0.937, mosaic=1.0

In [9]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!cp -r /content/mio_tcd_runs/ /content/drive/MyDrive/mio_tcd_runs